# Выполняет Волейко Андрей Владимирович
## Описание задания
База данных OpenPhish (https://openphish.com/) — это постоянно обновляемый архив структурированной и доступной для поиска информации обо всех фишинговых веб-сайтах, обнаруженных OpenPhish.
Кроме того, база данных содержит метаданные, которые можно использовать для обнаружения и анализа кибер-инцидентов, поиска шаблонов и тенденций или выступать в качестве набора данных для обучения или проверки для приложений ИИ.

In [1]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import random
import time
import pandas as pd

url_openphish = 'https://openphish.com/'

cycle_start_time = datetime.now() + timedelta(hours=3)

min_datetime = datetime(1, 1, 1)
need_to_create_csv = True
last_added = False
unique_targets = {}
unique_targets_count = 0
while True:

  page = requests.get(url_openphish, stream=True, allow_redirects=True, timeout=10, verify=False)
  soup = BeautifulSoup(page.text, "html.parser")
  table = soup.find('table', class_ = 'pure-table pure-table-striped')
  internal_table = table.find('tbody')

  alive_sites = []
  now = datetime.now()
  current_time = now.strftime("%m/%d/%Y %H:%M:%S")
  date = current_time.split(" ")[0]

  for tr in reversed(internal_table.find_all('tr')):
    url = ""
    target = ""
    row_time = ""
    row = []
    for td in tr.find_all('td'):
      row.append(td.text.strip())
    if row:
      url = row[0]
      target = row[1]
      row_time = date + " " + row[2]


      datetime_object = datetime.strptime(row_time, "%m/%d/%Y %H:%M:%S")
      if datetime_object > min_datetime:
        min_datetime = datetime_object
        last_added = False
      if ((now - datetime_object).total_seconds()/60-180<16) and datetime_object == min_datetime and not last_added:
        alive_sites.append(row)
        if unique_targets.get(target.removesuffix(' Inc.')) == None:
          unique_targets[target.removesuffix(' Inc.')] = 1
          unique_targets_count = unique_targets_count + 1
        else:
          unique_targets[target.removesuffix(' Inc.')] = unique_targets[target.removesuffix(' Inc.')] + 1
        last_added = True

  df = pd.DataFrame(alive_sites)
  if need_to_create_csv:
    df.to_csv('result.csv', mode='w', header=['URL', 'Target', 'Time'], index=False)
  else:
    df.to_csv('result.csv', mode='a', header=False, index=False)

  cycle_end_time = datetime.now() + timedelta(hours=3)
  # Вся нужная информация записывается в csv
  df_start_end = pd.DataFrame([cycle_end_time], [cycle_start_time])
  df_start_end.reset_index().to_csv('start_end_times.csv', mode='w', header=['Start time', 'End time'], index=False)

  unique_targets_list = []
  for key in sorted(unique_targets.keys(), key=unique_targets.get, reverse=True):
    unique_targets_list.append([key, unique_targets[key]])
  df_unique_targets = pd.DataFrame(unique_targets_list)
  df_unique_targets.reset_index().to_csv('unique_targets.csv', mode='w', header=['#', 'Target', 'Count'], index=False)

  df_unique_targets_count = pd.DataFrame([unique_targets_count])
  df_unique_targets_count.reset_index().to_csv('unique_targets_count.csv', mode='w', header=['#', 'Count'], index=False)

  need_to_create_csv = False
  time.sleep(300)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openphish.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openphish.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openphish.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: I

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')